# Improving the resolution of population coding by the use of multiple layers

Ricardo de Azambuja - CRNS - Plymouth University

## Abstract
A population code scheme as used in [Joshi and Maass, “Movement Generation with Circuits of Spiking Neurons.” and ???] simplifies the process of converting real values to a neural coding that can be applied to Spiking Neural Networks (SNN). This type of neural code has the special quality of keeping the energy contained in the input signal always the same among all the possible values avoiding the generation of a bias related to the number of spikes received. Notheless its simplicity it needs a large number of neurons if a fine scale is used. The human brain with its about 100 billion neurons could afford such an expensive scheme for a fine scaled code, but when a smaller animal as the fruit fly, with has approximatelly 100 thousand times less neurons, is analized, such a simple code could start to become a problem due to the need to precise movements as the ones necessary to fly and escape from a predator. Artificial implementations of neural network models also normally don't have enough computational resources to spend in a huge number of neurons in the input layer. The resolution, or the message space, can be greatly improved using the same number of neurons, but interpreting them as a multilayered system. The results from this work show that from an input layer made of 50 neurons instead of the same number of possible values as the number of neurons it's possible to have up to 100,000 values keeping the same energy profile and considerably improving the resolution of the system.

## Introduction

- The most classical citation of population code related with motor control: Georgopoulos, Kettner, and Schwartz, “Neuronal Population Coding of Movement Direction.”

### Explanations about the way the inputs are being coded using my multilayer population code


This type of code can generate 100.000 combinations versus 50 combinations a normal population code is able to generate using 50 neurons. Also with this multilayer population code it's easier to insert noise because there is a relationship between the position of the neuron and the value change associated to the neuron.  

In the case of the xdext variable, the input goes from neuron 0 up to neuron 49. The first 10 neurons (0 to 9) control the most significative digit and the last 10 neurons (40 to 49) control the last significative digit.  

Here I'm using a code a little bit different: instead of coding directly the number, I'm coding the position related to the range array of each variable. Even though I still have the same relationship between the digits, but now this relation is not 10x considering the final value. The 10x relation is with the index, but because the values are generated in a linear way, at the end it is 10x too with the initial value different from zero!  

**Population code takes too many neurons to get a good resolution:**  
Mixing with frequency/time(phase) could increase substantially the resolution, but the frequency is limited - at least - by the refractory period (between 2 and 3ms or 500 and 333.33Hz). Also in a simulation the size of the time step limits the maximum frequency in the input code.    
Looking for the brain waves frequencies, I've found that the Beta waves (between 16 and 31Hz) are associated with motor cortex and muscles activities (Baker 2007). Also the Gamma waves were observed to have frequencies from 40 up to 120Hz (Quyen et al. 2010). If I stick to the Beta waves because they have a direct relationship to motor cortex I would have another justification because membrane model (low-pass filter / integrator) is not going to filter out these frequencies. Considering the simulation time step, Nyquist–Shannon theorem tells us that the sampling must be at least twice as fast as the highest frequency. With a simulation time step of only 1ms the maximum frequency could be up to 500Hz. Right now I'm using a time step of 0.2ms, so the phase difference between two codes MUST be at least TWICE this value. The period of a 16Hz wave is 62.5ms, so to be able to distinguish between two different frequencies the system needs to wait 62.5ms until the possible second spike would arrive. This creates a barrier to the dynamic response of the system: the commands and feedbacks must have a maximum frequency smaller than 16Hz.
  Human reaction time to click a mouse button is, in average, 260ms (or about 4Hz).
  According to (Tim Wescott 2014): "The rule of thumb for sampled-time control systems is to choose a sampling rate equal to ten or twenty times your desired system bandwidth."  
**Another way to increase the resolution of a population code scheme** is creating more than one layer. The first one would be the biggest multiplier, the second a smaller, etc. As an example, if the first layer has 25 neurons, the fine distance between them could be represented by another layer with 25 neurons and now the resolution becomes $25*25=625$. If another layer is introduced with 25 neurons, it is possible to achieve an final resolution of $25*25*25=15625$. Using a unique layer with $25+25+25=75$ neurons, and each neuron representing a position, the maximum resolution would be of 75 points! A good thing about this idea is that the number of neurons that are spiking at the same time is kept constant if the first neuron of the finer layer has the value zero. 
  
  
  This two ideas are important to the implementation of a continuous learning scheme:
  
  http://en.wikipedia.org/wiki/Hick%27s_law
  
  http://en.wikipedia.org/wiki/Power_law_of_practice

In [1]:
import numpy

# Creates an array with the digits in a fixed (5) length
# The function can accept integers and 1D numpy.arrays.
# With integers it returns a 1D numpy.array
# With 1D numpy.arrays it returns a multiple lines numpy.array
def find_code(n):
    if type(n)==numpy.ndarray:
        ans = []
        for i in n:
            ans.append(numpy.array(list(str(i).zfill(5))).astype(dtype=int))
        return numpy.array(ans)
    else:
        return numpy.array(list(str(n).zfill(5))).astype(dtype=int)

In [2]:
find_code(65492), find_code(numpy.array([1,2,3]))

(array([6, 5, 4, 9, 2]), array([[0, 0, 0, 0, 1],
        [0, 0, 0, 0, 2],
        [0, 0, 0, 0, 3]]))

In [6]:
# The input (one big neurongroup with 300 neurons) will be divided like this:
# 6 groups of 50 neurons.
# And each group of 50 neurons is divided in 5 groups of 10 neurons.
# Therefore each digit will be encoded in 10 neurons.
# Example: 
# xdest=12345 => [1,12,23,34,45]
# ydest=00324 => [50,60,73,82,94]
#
# With this type of code the idea of "neuron more significative" is implemented and
# becomes a lot easier to insert a small amount of noise into the system.
#
# - Group 1: xdest => main offset:0
# - Group 2: ydest => main offset:50
# - Group 3: teta1 => main offset:100
# - Group 4: teta2 => main offset:150
# - Group 5: tau1  => main offset:200
# - Group 6: tau2  => main offset:250
# 
# find_code(number=12345) simply creates an numpy.array([1,2,3,4,5]) and the codification is actually
# done by the offset of numpy.arange(init,final,10)

xdest_lsm = find_code(xdest_idx) + numpy.arange(0,50,10)
ydest_lsm = find_code(ydest_idx) + numpy.arange(50,100,10)
teta1_lsm = find_code(joint_states_idx[:,0]) + numpy.arange(100,150,10)
teta2_lsm = find_code(joint_states_idx[:,1]) + numpy.arange(150,200,10)
tau1_lsm = find_code(torques_normalized_idx[:,0]) + numpy.arange(200,250,10)
tau2_lsm = find_code(torques_normalized_idx[:,1]) + numpy.arange(250,300,10)

#
# At the end this is the same idea I had before where I always made sure a fixed number of neurons spiked 
# for each code representation.

In [9]:
numpy.arange(0,50,10)

array([ 0, 10, 20, 30, 40])

In [7]:
find_code(54321) + numpy.arange(0,50,10)

array([ 5, 14, 23, 32, 41])

In [10]:
numpy.arange(50,100,10)

array([50, 60, 70, 80, 90])

In [11]:
find_code(54321) + numpy.arange(50,100,10)

array([55, 64, 73, 82, 91])